In [1]:
import cv2
import os
from tqdm import tqdm

Arguments for running the script. Please make sure to set these correctly

In [2]:
imdir = "C:/Users/dieck/Desktop/larger_test_set/" #Input Image directory
imFilePath = "" #File containing image paths
labeling_Person = "Simon Dieck" #Person who does the labeling, to check for bias
fileLabelsPath = "macula_occ_disk_lables" # Output file path
startLine = 0 #Line at which you start in the input file

mode = "area" #"center" #TODO

refPt = [] #Global variable

Event Handler

In [3]:
def mark_area(event, x, y, flags, param):
    global refPt
    
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt.append((x,y))
        
    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x,y))
        
        if len(refPt) == 4:
            cv2.rectangle(image, refPt[2], refPt[3], (0, 255, 0), 2)
        else:
            cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
            
        cv2.imshow("fundoscopy", image)

Labeling code.

Use "r" to redo your labeling

Use "spacebar" to go to the next picture

Use "q" to quit the program

In [ ]:
imFile = open(imFilePath, 'r')
fileLabels = open(fileLabelsPath, 'a')

fileLabels.write("Origin Folder: {} \n Labeled by: {} \n".format(imdir, labeling_Person))

cleanup = False

cv2.namedWindow("fundoscopy")

for cnt, path in tqdm(enumerate(imFile)):
    if cnt >= startLine:
        fileLabels.write(path+": ")
        image = cv2.imread(imdir + path)
        clone = image.copy()
        cv2.setMouseCallback("fundoscopy", mark_area)
        
        while True:
            cv2.imshow("fundoscopy", image)
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord("r"):
                image = clone.copy()
                refPt = []
                
            if key == ord("q"):
                cleanup = True
                break
                
            if key == 32:
                break
        
        if cleanup:
            fileLabels.write("Interrupted at line: "+cnt+"\n")
            break
        if len(refPt) >= 2:
            fileLabels.write("Optic Disk: "+str(refPt[0])+","+str(refPt[1])+" ")
            if len(refPt) == 4:
                fileLabels.write("Fovea: "+str(refPt[2])+","+str(refPt[3]))
        fileLabels.write("\n")
        refPt = []
    
imFile.close()
fileLabels.close()
cv2.destroyAllWindows()